# Quadkeys
Quadkeys are a useful way to identify tiles in a tile-based map system. They consist of a sequence of characters that encode the tile’s location and zoom level in a hierarchical way. Each character represents a quadrant of the map at a given zoom level, with the left-most character representing the top-left quadrant.

In [1]:
!activate InputOutputMappingEnv

In [2]:
#!pip uninstall rioxarray geopy mercantile --y
#!pip install rioxarray geopy mercantile
#!pip install numpy==24
#!pip install rasterio
#!pip install fiona

In [1]:
#!which numpy 

In [45]:
import os
import mercantile
#import gdal
import numpy as np
import mercantile,fiona
import rasterio as rio
import rioxarray 
from rasterio import mask as msk
import random
import geopy.distance
#import os, osr
#import geopandas as gpd
import shutil
import pickle

### Change the CRS projection

### CRS conversion with exact spatial resolution

In [101]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
def CRS_conversion_exact(input_tif,output_tif,dst_crs = 'EPSG:4326'):
    # Input and output file paths
    # input_tif = '/s/chopin/e/proj/hyperspec/Abdul/Data/gSSURGO/gSSURGO-mukey_202310_CA.TIF'  # Path to your input GeoTIFF
    # output_tif =   '/s/chopin/e/proj/hyperspec/Abdul/Data/gSSURGO/gSSURGO-mukey_202310_CA_CRS_4326_3.TIF'  # Path to save the reprojected GeoTIFF

    # Open the input raster
    with rasterio.open(input_tif) as src:
        # Define the target CRS (EPSG:4326)
        dst_crs = 'EPSG:4326'

        # Calculate the transform for the target CRS, but keep the original resolution
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)

        # Calculate the new resolution (pixel size) to match the original resolution
        original_resolution = (src.transform[0], src.transform[4])  # pixel width, pixel height

        ##Adjust the transform to maintain the original resolution
        new_transform = rasterio.Affine(
            original_resolution[0]/100000, transform.b, transform.c,
            transform.d, original_resolution[1]/100000, transform.f
        )
        
        # Update the width and height to reflect the new bounds at the same resolution
        width = int((src.bounds.right - src.bounds.left) / original_resolution[0])
        height = int((src.bounds.top - src.bounds.bottom) / abs(original_resolution[1]))

        # Update the metadata with the new CRS, transform, and dimensions
        out_meta = src.meta.copy()
        out_meta.update({
            'crs': dst_crs,
            'transform': new_transform,
            'width': width,
            'height': height

        })

        # Reproject and save the reprojected image
        with rasterio.open(output_tif, 'w', **out_meta) as dest:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dest, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=new_transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest
                )

    print("Reprojection complete, with the same spatial resolution maintained.")



#### CRS conversion without exact spatial resolution

In [89]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
def CRS_conversion(input_tif,output_tif,dst_crs = 'EPSG:4326'):
    # Input and output file paths
    # input_tif = '/s/chopin/e/proj/hyperspec/Abdul/Data/gSSURGO/gSSURGO-mukey_202310_CA.TIF'  # Path to your input GeoTIFF
    # output_tif =   '/s/chopin/e/proj/hyperspec/Abdul/Data/gSSURGO/gSSURGO-mukey_202310_CA_CRS_4326_2.TIF'  # Path to save the reprojected GeoTIFF

    # Open the input raster
    with rasterio.open(input_tif) as src:
        # Define the target CRS (EPSG:4326)


        # Calculate the transform and new dimensions for the target CRS
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        
        original_resolution = (src.transform[0], src.transform[4])  # pixel width, pixel height

        width = int((src.bounds.right - src.bounds.left) / original_resolution[0])
        height = int((src.bounds.top - src.bounds.bottom) / abs(original_resolution[1]))

        # Update the metadata with the new CRS, transform, and dimensions
        out_meta = src.meta.copy()
        out_meta.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # Reproject and save the reprojected image
        with rasterio.open(output_tif, 'w', **out_meta) as dest:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dest, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest
                )

    print("Reprojection complete.")


### Not preferred method

In [48]:
def changeCRS(input_tif_path,output_tif_path = None,to_crs = "EPSG:4326"):
    # input_tif = 'path of input tif file'
    # output_tif = 'path of output tif file'
    rds = rioxarray.open_rasterio(input_tif_path)
    rds_4326 = rds.rio.reproject(to_crs)
    rds_4326.rio.to_raster(output_tif_path)
    return rds_4326

In [48]:
def changeCRS2(input_tif_path,output_tif_path = None,to_crs = "EPSG:4326"):
    # input_tif = 'path of input tif file'
    # output_tif = 'path of output tif file'
    rds = rioxarray.open_rasterio(input_tif_path)
    rds_4326 = rds.rio.reproject(to_crs)
    rds_4326.rio.to_raster(output_tif_path)
    return rds_4326

#### Read all the folder and file

In [49]:
import os
def read_all_file_folder(rootDir):

    # Path to the directory containing the folders
    directory_path = rootDir

    # List all folder names in the directory
    folder_names = [name for name in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, name))]

    # Print the folder names
    # for folder in folder_names:
    #     print(folder)
    return folder_names

### Get the bounding box

In [50]:
def get_bounding_box(filePath):
    if filePath is not None:
        raster = rasterio.open(filePath)
        return raster.bounds
    return None,None

In [51]:
def get_quad_tile(lat, lon, precision):
    ret = mercantile.tile(lon,lat,precision)
    return ret

def get_quad_key_from_tile(x, y, zoom):
    return mercantile.quadkey(x, y, zoom)

def get_quad_key_from_tile(tile):
    return mercantile.quadkey(tile.x, tile.y, tile.z)

def get_tile_from_key(key):
    return mercantile.quadkey_to_tile(key)

def get_quad_key(lat, lon, zoom):
    tile = get_quad_tile(lat, lon, precision=zoom)
    # print(tile)
    return get_quad_key_from_tile(tile.x, tile.y, tile.z)

def get_max_possible_xy(zoom):
    if zoom == 0:
        return 0
    return 2**zoom-1

def validate_tile(tile):
    max_xy = get_max_possible_xy(tile.z)
    if tile.x > max_xy or tile.x < 0 or tile.y > max_xy or tile.y < 0:
        return False
    return True

#GIVEN A QUAD_TILE, GET ITS LAT-LNG BOUNDS
def get_bounding_lng_lat(tile_key):
    tile = get_tile_from_key(tile_key)
    bounds = mercantile.bounds(tile)
    # print("ul: " , ul)
    return [bounds.west, bounds.east, bounds.north, bounds.south]

# Get all the tiles from bounds and zoom level 
# bounds: (l = left/west, b = bottom/south, r = right/east, t = top/north)
def get_all_tiles_for_a_bounds(bounds,zoom_level):
    all_tiles = mercantile.tiles(*bounds, zooms=[zoom_level])
    return all_tiles

# provide a root directory with folders named by quadkeys and return a list of quadkeys
def read_all_folders_as_quadkey(path):
    
    sub_dirs = [name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]
    #print(sub_folders)
    #print(f"total folders/quadkeys: {len(sub_dirs)}")

    #sub dirs/folders named by quadkeys
    allQuadKeys = sub_dirs
    return allQuadKeys

#provide a list of quadkeys and state quadkeys dictionary and return the statewise splitting dictionary for the provided list
def split_quadkeys_to_each_state(allQuadKeys,state_all_quadkeys):
    #Counting SMAP downloaded kyes for each state
    smap_statewise_quadkeys = {}

    for qk in allQuadKeys:
        #print(type(qk))
        for state in state_all_quadkeys.keys():
            if qk in state_all_quadkeys[state]:
                #print("yes")
                if state not in smap_statewise_quadkeys.keys():
                    smap_statewise_quadkeys[state] = [qk]
                else:
                    smap_statewise_quadkeys[state].append(qk)
    return smap_statewise_quadkeys
            

### Generate all the Quadkeys from lat lon bounding box

In [52]:
def get_all_quadKeys(bounds,zoomLevel = 10):

    all_tiles = mercantile.tiles(*bounds, zooms=[zoomLevel])
    quadkeys = []
    for t in all_tiles:
        #print(get_quad_key_from_tile(t))
        quadkeys.append(get_quad_key_from_tile(t))
    
    #with open('quadkeys.pkl', 'wb') as fp:
    #    pickle.dump(quadkeys, fp)
    return quadkeys

### Quadkey to Lat Lon bounds

In [53]:
def getLatLonBoundsFromQKeys(quadKeyList,zoomlevel = 10):
    allbbox = {}
    for qk in quadKeyList:
        #print(qk)
        tile = mercantile.quadkey_to_tile(qk)
        bbox = mercantile.bounds(tile) #l,b,r,t = mercantile.bounds(385, 820, 11) # l = left/west, b = bottom/south, r = right/east, t = top/north
        allbbox[qk] = bbox
    print(f"total bbox:  {len(allbbox)}")
    #print(allbbox)
    return allbbox

### Split large Tif by BBox

In [112]:
def split_large_tif_by_bbox(bboxDict,LargeTifPath = None, outDir = None,limit = None):
    cnt = 0
    for k,v in bboxDict.items():    
        print('now processing Qkey: ',k,v.west, v.south, v.east, v.north)
        west=v.west
        south=v.south
        east=v.east
        north=v.north
        #outPath = r"C:\\Summer_work\\Soil_type\\Polaris\\AOI12\\Clay_Smaller_tiffs_AOI12"
        outpath =outDir+"/"+k+".TIF"
        gdal_translate_cmd = f"gdal_translate -projwin {west} {north} {east} {south} {LargeTifPath} {outpath}"
        os.system(gdal_translate_cmd)
        cnt+=1
        if limit is not None and cnt>=limit:
            break
    print( " Done .. ")

### Read the file Properties

## ENMAP

In [68]:
raw_tif_filePath = '/s/chopin/e/proj/hyperspec/EnMAP_Collected_datasets/AOI1/ENMAP01-____L2A-DT0000007955_20230218T191613Z_013_V010401_20240204T092653Z-SPECTRAL_IMAGE.TIF'
raster = rio.open(raw_tif_filePath)
polaris_CRS = raster.crs
print(polaris_CRS)

EPSG:32610


In [69]:
raster.profile

{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -32768.0, 'width': 1235, 'height': 1156, 'count': 224, 'crs': CRS.from_epsg(32610), 'transform': Affine(30.0, 0.0, 751515.0,
       0.0, -30.0, 3997185.0), 'blockysize': 3, 'tiled': False, 'compress': 'deflate', 'interleave': 'band'}

In [137]:
# input_tif = '/s/chopin/e/proj/hyperspec/Abdul/Data/gSSURGO/gSSURGO-mukey_202310_CA.TIF'  # Path to your input GeoTIFF
output_tif =   '/s/chopin/e/proj/hyperspec/Abdul/tmp/ENMAP_AOI1_EPSG4326_.TIF'  # Path to save the reprojected GeoTIFF
#CRS_conversion_exact(raw_tif_filePath,output_tif)

In [138]:
raster = rio.open(output_tif)
print(raster.crs)

EPSG:4326


In [139]:
raster.profile

{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -32768.0, 'width': 1393, 'height': 1064, 'count': 224, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0003029821278506558, 0.0, -120.21759721521599,
       0.0, -0.0003029821278506558, 36.086765086098154), 'blockysize': 1, 'tiled': False, 'interleave': 'pixel'}

In [141]:
bounds = raster.bounds
print(bounds)

BoundingBox(left=-120.21759721521599, bottom=35.76439210206506, right=-119.79554311112003, top=36.086765086098154)


In [103]:
raster2 = rio.open(output_tif)
print(raster2.crs)

EPSG:4326


In [104]:
raster2.profile

{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -32768.0, 'width': 1235, 'height': 1156, 'count': 224, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0003, 0.0, -120.21759721521599,
       0.0, -0.0003, 36.086765086098154), 'blockysize': 1, 'tiled': False, 'interleave': 'pixel'}

In [105]:
bounds = raster2.bounds
print(bounds)

BoundingBox(left=-120.21759721521599, bottom=35.73996508609815, right=-119.84709721521598, top=36.086765086098154)


In [142]:
z_level = 14
qkeys = get_all_quadKeys(bounds=bounds,zoomLevel=z_level)
len(qkeys)

380

In [143]:
bBox_dict = getLatLonBoundsFromQKeys(qkeys,zoomlevel=z_level)
len(bBox_dict)

total bbox:  380


380

In [144]:
for box in bBox_dict.items():
    print(box)
    break

('02301210122200', LngLatBbox(west=-120.234375, south=36.084621296069315, east=-120.21240234375, north=36.102376448736436))


In [149]:
Ssurgo_path = '/s/chopin/e/proj/hyperspec/Abdul/Data/gSSURGO/gSSURGO-mukey_202310_CA_CRS_4326_2.TIF'  # Path to save the reprojected GeoTIFF
raster = rio.open(Ssurgo_path)
print(raster.crs)

EPSG:4326


In [136]:
raster.profile

{'driver': 'GTiff', 'dtype': 'uint32', 'nodata': 2147483647.0, 'width': 31333, 'height': 21329, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00032045201503583416, 0.0, -123.27511604750066,
       0.0, -0.00032045201503583416, 37.90459986684897), 'blockysize': 1, 'tiled': False, 'interleave': 'band'}

In [155]:
Polaris_path = '/s/chopin/e/proj/hyperspec/AOI1/AOI1_clay.tif'
raster = rio.open(Polaris_path)
print(raster.crs)

EPSG:4326


In [156]:
raster.profile

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -9999.0, 'width': 1643, 'height': 1282, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00027777777777515233, 0.0, -120.22250000000733,
       0.0, -0.00027777777777515666, 36.09611111111964), 'blockysize': 1, 'tiled': False, 'interleave': 'band'}

In [157]:
LargeTifPath = Polaris_path#Ssurgo_path#output_tif
outDir =   '/s/chopin/e/proj/hyperspec/Abdul/tmp/POLARIS_AOI1_EPSG4326' #ENMAP_AOI1_EPSG4326
if not os.path.exists(outDir):
    os.mkdir(outDir)
split_large_tif_by_bbox(bboxDict=bBox_dict,LargeTifPath=LargeTifPath,outDir=outDir,limit = 5)

now processing Qkey:  02301210122200 -120.234375 36.084621296069315 -120.21240234375 36.102376448736436
Input file size is 1643, 1282
0...10...20...30...40...50...60...70...80...90...100 - done.
now processing Qkey:  02301210122202 -120.234375 36.06686213257887 -120.21240234375 36.084621296069315
Input file size is 1643, 1282
0...10...20...30...40...50...60...70...80...90...100 - done.
now processing Qkey:  02301210122220 -120.234375 36.04909895906564 -120.21240234375 36.06686213257887
Input file size is 1643, 1282
0...10...20...30...40...50...60...70...80...90...100 - done.
now processing Qkey:  02301210122222 -120.234375 36.03133177633188 -120.21240234375 36.04909895906564


Warning 1: Computed -srcwin -43 -23 79 64 falls partially outside raster extent. Going on however.
Warning 1: PROJ: proj_create_from_database: Open of /usr/local/anaconda3/2023.03/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /usr/local/anaconda3/2023.03/share/proj failed
Warning 1: Computed -srcwin -43 41 79 64 falls partially outside raster extent. Going on however.
Warning 1: PROJ: proj_create_from_database: Open of /usr/local/anaconda3/2023.03/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /usr/local/anaconda3/2023.03/share/proj failed
Warning 1: Computed -srcwin -43 105 79 64 falls partially outside raster extent. Going on however.
Warning 1: PROJ: proj_create_from_database: Open of /usr/local/anaconda3/2023.03/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /usr/local/anaconda3/2023.03/share/proj failed
Warning 1: Computed -srcwin -43 169 79 64 falls partially outside raster extent. Going on however.
Warning 1: 

Input file size is 1643, 1282
0...10...20...30...40...50...60...70...80...90...100 - done.
now processing Qkey:  02301210300000 -120.234375 36.01356058518154 -120.21240234375 36.03133177633188
Input file size is 1643, 1282
0...10...20...30...40...50...60...70...80...90...100 - done.
 Done .. 


Warning 1: Computed -srcwin -43 233 79 64 falls partially outside raster extent. Going on however.
Warning 1: PROJ: proj_create_from_database: Open of /usr/local/anaconda3/2023.03/share/proj failed
Warning 1: PROJ: proj_create_from_database: Open of /usr/local/anaconda3/2023.03/share/proj failed


In [147]:
small_tif_path = outDir+'/'+'02301210300000.TIF'
raster = rio.open(small_tif_path)
polaris_CRS = raster.crs
print(polaris_CRS)

EPSG:4326


In [148]:
raster.profile

{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -32768.0, 'width': 73, 'height': 59, 'count': 224, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0003029821278506558, 0.0, -120.23456421437562,
       0.0, -0.0003029821278506558, 36.03162233882934), 'blockysize': 1, 'tiled': False, 'interleave': 'pixel'}

In [151]:
small_tif_path = outDir+'/'+'02301210300000.TIF'
raster = rio.open(small_tif_path)
polaris_CRS = raster.crs
print(polaris_CRS)

EPSG:4326


In [152]:
raster.profile

{'driver': 'GTiff', 'dtype': 'uint32', 'nodata': 2147483647.0, 'width': 69, 'height': 55, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00032045201503583416, 0.0, -120.23466732884067,
       0.0, -0.00032045201503583416, 36.03155783896452), 'blockysize': 29, 'tiled': False, 'interleave': 'band'}

In [158]:
small_tif_path = outDir+'/'+'02301210300000.TIF'
raster = rio.open(small_tif_path)
polaris_CRS = raster.crs
print(polaris_CRS)

EPSG:4326


In [159]:
raster.profile

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -9999.0, 'width': 79, 'height': 64, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00027777777777515233, 0.0, -120.23444444445165,
       0.0, -0.00027777777777515666, 36.03138888889803), 'blockysize': 25, 'tiled': False, 'interleave': 'band'}

### Generate Quadkeys from a large image

In [ ]:
# read large file and get the quadkeys for aligned zoom level
#get_all_quadKeys()

In [7]:
rootDir = '/s/chopin/e/proj/hyperspec/Tanjim/EnMap-Soil-Texture/vitmae/dataset/219-channels/tiffs/'
all_folders = read_all_file_folder(rootDir)
len(all_folders)

888

In [8]:
all_folders[100]

'tile-3019'

In [9]:
folder = all_folders[0]
tile_path = rootDir+folder+'/'+folder+'.TIF'
raster = rio.open(tile_path)
polaris_CRS = raster.crs
print(polaris_CRS)

EPSG:4326


## Map ENMAP small Tile to SSURGO or POLARIS

In [13]:
outFolder = '/s/chopin/e/proj/hyperspec/Abdul/Data/gSSURGO/small_tifs/mapped_with_ENMAP'
if not os.path.exists(outFolder):
    os.mkdir(outFolder)

In [26]:
import rasterio
from rasterio.windows import from_bounds

large_tif = '/s/chopin/e/proj/hyperspec/Abdul/Data/gSSURGO/gSSURGO-mukey_202310_CA_CRS_4326_3.TIF'  # Path to the large input GeoTIFF

for folder in all_folders:
    tile_path = rootDir+folder+'/'+folder+'.TIF'

    # File paths
    small_tif = tile_path  # Path to the smaller GeoTIFF (used to get the bounding box)
    output_tif = outFolder+'/'+folder+'.TIF'  # Path to save the cropped GeoTIFF

    # Open the small TIFF to get its bounding box
    with rasterio.open(small_tif) as small_src:
        # Get the bounding box in the coordinate system of the small TIFF
        minx, miny, maxx, maxy = small_src.bounds

    # Open the large TIFF
    with rasterio.open(large_tif) as large_src:
        # Create a window using the bounding box from the small TIFF
        window = from_bounds(minx, miny, maxx, maxy, transform=large_src.transform)

        # Read the data within the window from the large TIFF
        data = large_src.read(window=window)

        # Update metadata to reflect the new window size
        out_meta = large_src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": window.height,
            "width": window.width,
            "transform": large_src.window_transform(window)
        })

        # Write the cropped data to a new file
        with rasterio.open(output_tif, "w", **out_meta) as dest:
            dest.write(data)
    #break
print("Cropping complete.")


Cropping complete.


In [21]:
filePath = '/s/chopin/e/proj/hyperspec/Abdul/Data/POLARIS/lat3637_lon-120-119.tif'
raster = rio.open(filePath)
polaris_CRS = raster.crs
print(polaris_CRS)

EPSG:4326


In [22]:
raster.profile

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -9999.0, 'width': 3600, 'height': 3600, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00027777777777515666, 0.0, -120.0,
       0.0, -0.00027777777777515666, 37.0), 'blockysize': 1, 'tiled': False, 'compress': 'deflate', 'interleave': 'band'}

In [23]:
data = raster.read()
data.shape

(1, 3600, 3600)

In [24]:
data.min(),data.max()

(-9999.0, 92.25953)

In [25]:
data[0][:10]

array([[56.414062, 56.66813 , 69.68084 , ..., 74.68768 , 74.68768 ,
        74.68768 ],
       [58.55963 , 59.360786, 48.355186, ..., 74.68768 , 74.68768 ,
        74.68768 ],
       [62.2182  , 59.16927 , 49.202255, ..., 74.68768 , 74.68768 ,
        74.68768 ],
       ...,
       [49.73828 , 70.46387 , 70.23366 , ..., 74.68768 , 74.68768 ,
        56.6714  ],
       [69.509735, 54.68591 , 54.68591 , ..., 74.68768 , 74.68768 ,
        74.68768 ],
       [50.457886, 49.63951 , 50.033268, ..., 74.68768 , 74.68768 ,
        74.68768 ]], dtype=float32)

In [20]:
import pandas as pd
file = '/s/chopin/e/proj/hyperspec/Abdul/Data/gSSURGO/FY2024_gSSURGO_Tabular_CSV/chorizon.csv'
df = pd.read_csv(file,nrows=100)
df.head()

,OID_,hzname,desgndisc,desgnmaster,desgnmasterprime,desgnvert,hzdept_l,hzdept_r,hzdept_h,hzdepb_l,...,ph2osoluble_l,ph2osoluble_r,ph2osoluble_h,ptotal_l,ptotal_r,ptotal_h,excavdifcl,excavdifms,cokey,chkey
0,1,C,NaN,C,NaN,2.0,NaN,20,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24629637,73203533
1,2,A,NaN,A,NaN,1.0,NaN,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24629637,73203532
2,3,C1...C5,NaN,C,NaN,2.0,NaN,20,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24630059,73204462
3,4,Ap,NaN,A,NaN,1.0,NaN,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24630059,73204461
4,5,Bt,NaN,B,NaN,2.0,NaN,13,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24630061,73204464


## ENMAP

In [4]:
filePath = '/s/chopin/e/proj/hyperspec/Tanjim/EnMap-Soil-Texture/vitmae/dataset/219-channels/tiffs/tile-1648/tile-1648.TIF'
raster = rio.open(filePath)
enmap_raw_CRS = raster.crs
enmap_raw_CRS

(CRS.from_epsg(32610), CRS.from_epsg(4326))

In [ ]:
raster.bounds

In [ ]:
raster.profile

## State Lat Lon Bounds

In [12]:
#State lat lon bounds: (l = left/west, b = bottom/south, r = right/east, t = top/north)
California_bounds = (-124.41060660766607,32.5342307609976,-114.13445790587905,42.00965914828148) #California lat lon bounds
Colorado_bounds = (-109.05919619986199,36.99275055519555,-102.04212644366443,41.00198213121131) #Colorado
Arkansas_bounds = (-94.61946646626465,33.00413641175411,-89.65547287402873,36.49965029279292)
Texas_bounds = (-106.64719063660635,25.840437651866516,-93.5175532104321,36.50050935248352)
Wyoming_bounds = (-111.05843295392954,40.995109653686534,-104.05213107971079,45.006059349083486)
Oklahoma_bounds = (-103.00405723377233,33.61664597114971,-94.43282317863178,37.002200211792115)
NewMaxico_bounds = (-109.04842831788318,31.332406253852533,-103.0004679397794,37.00048209241092)

state_all_quadkeys = {}

### === xxx ====

In [17]:
state_all_quadkey.keys()

dict_keys(['California_bounds', 'Colorado_bounds', 'Texas_bounds', 'Arkansas_bounds', 'Wyoming_bounds', 'Oklahoma_bounds', 'NewMaxico_bounds'])

In [20]:
# Saving the quadkeys for each state (SMAP data)
save = True
if save: 
    with open('state_quadKey_z13_list.pkl', 'wb') as fp:
        pickle.dump(state_all_quadkeys, fp)

In [26]:
# Reading all statewise quadkeys
file = open('state_quadKey_z13_list.pkl','rb')
state_all_quadkeys = pickle.load(file)
state_all_quadkeys.keys()

dict_keys(['California', 'Colorado', 'Arkansas', 'Texas', 'Wyoming', 'Oklahoma', 'NewMaxico'])

In [28]:
state_all_quadkeys['Colorado_bounds']

['0213223223332',
 '0231001001110',
 '0231001001112',
 '0231001001130',
 '0231001001132',
 '0231001001310',
 '0231001001312',
 '0231001001330',
 '0231001001332',
 '0231001003110',
 '0231001003112',
 '0231001003130',
 '0231001003132',
 '0231001003310',
 '0231001003312',
 '0231001003330',
 '0231001003332',
 '0231001021110',
 '0231001021112',
 '0231001021130',
 '0231001021132',
 '0231001021310',
 '0231001021312',
 '0231001021330',
 '0231001021332',
 '0231001023110',
 '0231001023112',
 '0231001023130',
 '0231001023132',
 '0231001023310',
 '0231001023312',
 '0231001023330',
 '0231001023332',
 '0231001201110',
 '0231001201112',
 '0231001201130',
 '0231001201132',
 '0231001201310',
 '0231001201312',
 '0231001201330',
 '0231001201332',
 '0231001203110',
 '0231001203112',
 '0231001203130',
 '0231001203132',
 '0231001203310',
 '0231001203312',
 '0231001203330',
 '0231001203332',
 '0231001221110',
 '0231001221112',
 '0231001221130',
 '0231001221132',
 '0231001221310',
 '0231001221312',
 '02310012

## All Quadkeys (length 13) for States: CA, CO, AK, TX, OK, NM

In [18]:
# total number of quadkeys for each state
for k in state_all_quadkeys.keys():
    print(f" {k} = {len(state_all_quadkeys[k])}")

 California_bounds = 64155
 Colorado_bounds = 18880
 Texas_bounds = 85215
 Arkansas_bounds = 11172
 Wyoming_bounds = 20286
 Oklahoma_bounds = 18620
 NewMaxico_bounds = 21823


## All Quadkeys (length 11) for States: CA, CO, AK, TX, OK, NM

In [27]:
# total number of quadkeys for each state
for k in state_all_quadkeys.keys():
    print(f" {k} = {len(state_all_quadkeys[k])}")

 California = 4071
 Colorado = 1271
 Arkansas = 725
 Texas = 5400
 Wyoming = 1312
 Oklahoma = 1176
 NewMaxico = 1440


## SMAP

### Reading all the SMAP data folders as quadkeys

In [21]:
smap_path = '/s/chopin/f/proj/fineET/data_for_soil_moisture_work/new_quadhash_all_datasets/smap_3km_preprocessed_202021/split'

folder = smap_path

sub_folders = [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]

#print(sub_folders)
print(f"total folders {len(sub_folders)}")

#sub folder named after each quadkey
allQuadKeys = sub_folders

total folders 19200


In [27]:
if '0231010111130' in allQuadKeys:
    print ("yes")

yes


In [29]:
if '0231001001112' in allQuadKeys:
    print ("yes")

### converting each quadkey to Lat Lon bounds

In [20]:
allbbox = {}
zoomlevel = 11
for qk in allQuadKeys:
    #print(qk)
    tile = mercantile.quadkey_to_tile(qk)
    bbox = mercantile.bounds(tile) #l,b,r,t = mercantile.bounds(385, 820, 11) # l = left/west, b = bottom/south, r = right/east, t = top/north
    allbbox[qk] = bbox
print(f"total bbox:  {len(allbbox)}")

total bbox:  14740


In [30]:
#Counting SMAP downloaded kyes for each state
smap_statewise_quadkeys = {}

for qk in allQuadKeys:
    #print(type(qk))
    for state in state_all_quadkeys.keys():
        if qk in state_all_quadkeys[state]:
            #print("yes")
            if state not in smap_statewise_quadkeys.keys():
                smap_statewise_quadkeys[state] = [qk]
            else:
                smap_statewise_quadkeys[state].append(qk)
            
#state_all_quadkeys


In [31]:
smap_statewise_quadkeys.keys()

dict_keys(['Colorado_bounds', 'NewMaxico_bounds', 'Wyoming_bounds', 'Oklahoma_bounds'])

## SMAP Downloaded Quadkeys for each state

In [32]:
# SMAP Data: total number of quadkeys for each state
for k in smap_statewise_quadkeys.keys():
    print(f" {k} = {len(smap_statewise_quadkeys[k])}")

 Colorado_bounds = 18762
 NewMaxico_bounds = 414
 Wyoming_bounds = 114
 Oklahoma_bounds = 69


In [34]:
smap_statewise_quadkeys_CO = smap_statewise_quadkeys['Colorado_bounds']
smap_statewise_quadkeys_CO

['0231013013320',
 '0231013021132',
 '0231010101023',
 '0231012313231',
 '0231012201321',
 '0231010330312',
 '0231011103000',
 '0231012010112',
 '0231012001122',
 '0231011103300',
 '0231010230230',
 '0231011013001',
 '0231003133331',
 '0231003213012',
 '0231011001303',
 '0231011222003',
 '0231010130300',
 '0231003300332',
 '0231012103310',
 '0231013200013',
 '0231012221003',
 '0231011211210',
 '0231001230220',
 '0231011301031',
 '0231001030121',
 '0231003012002',
 '0231012130131',
 '0231010211313',
 '0231001303233',
 '0231010101012',
 '0231011223220',
 '0231010202033',
 '0231001121203',
 '0231003312120',
 '0231013032210',
 '0231010202333',
 '0231013210100',
 '0231010201122',
 '0231001113000',
 '0231012331001',
 '0231013301123',
 '0231003102213',
 '0231010210012',
 '0231010232323',
 '0231010002203',
 '0231012330311',
 '0231001032311',
 '0231012300023',
 '0231010201200',
 '0231011221301',
 '0231011210220',
 '0231013000220',
 '0231010111000',
 '0231012122302',
 '0231011002032',
 '02310111

In [33]:
# Saving the quadkeys for each state (SMAP data)

save = False
if save: 
    with open('SMAP_statewise_Colorado_quadKey_z13_list.pkl', 'wb') as fp:
        pickle.dump(smap_statewise_quadkeys, fp)

## GRIDMET

In [35]:
# Getting all the folders as quadkey list
path = '/s/chopin/f/proj/fineET/data_for_soil_moisture_work/new_quadhash_all_datasets/gridmet/split'
all_GridMET_quadKeys = read_all_folders_as_quadkey(path)

In [36]:
print(f" total quadkeys: {len(all_GridMET_quadKeys)}")

 total quadkeys: 19200


In [37]:
# Splitting quadkeys to each state
GridMET_statewise_quadkeys = split_quadkeys_to_each_state(all_GridMET_quadKeys,state_all_quadkeys)

In [39]:
# Saving the quadkeys for each state (SMAP data)
save = False
if save: 
    with open('GridMET_statewise_quadKey_z13_list.pkl', 'wb') as fp:
        pickle.dump(GridMET_statewise_quadkeys, fp)

## GRIDMET Downloaded Quadkeys for each state

In [38]:
# GridMET Data: total number of quadkeys for each state
for k in GridMET_statewise_quadkeys.keys():
    print(f" {k} = {len(GridMET_statewise_quadkeys[k])}")

 Colorado_bounds = 18762
 NewMaxico_bounds = 414
 Wyoming_bounds = 114
 Oklahoma_bounds = 69


## gNatsGo

In [40]:
path = '/s/chopin/f/proj/fineET/data_for_soil_moisture_work/new_quadhash_all_datasets/gNATSGO_preprocessed_202021/split'
# Getting all the folders as quadkey list
all_gNatsGo_quadKeys = read_all_folders_as_quadkey(path)

In [45]:
print(f" total quadkeys: {len(all_gNatsGo_quadKeys)}")

 total quadkeys: 12226


In [30]:
#all_gNatsGo_quadKeys

In [46]:
# Splitting quadkeys to each state
gNatsGo_statewise_quadkeys = split_quadkeys_to_each_state(all_gNatsGo_quadKeys,state_all_quadkeys)

In [32]:
#state_all_quadkeys

In [48]:
# Saving the quadkeys for each state (gNatsGo data)
save = False
if save: 
    with open('gNatsGo_statewise_quadKey_z13_list.pkl', 'wb') as fp:
        pickle.dump(gNatsGo_statewise_quadkeys, fp)

## gNatsGo Downloaded Quadkeys for each state

In [47]:
# gNatsGo Data: total number of quadkeys for each state
for k in gNatsGo_statewise_quadkeys.keys():
    print(f" {k} = {len(gNatsGo_statewise_quadkeys[k])}")

 Colorado_bounds = 12024
 NewMaxico_bounds = 303
 Wyoming_bounds = 102


In [11]:
gNatsGo_statewise_quadkeys

{'Colorado': ['02310121200',
  '02310122032',
  '02310011022',
  '02310102312',
  '02310103322',
  '02310102131',
  '02310101122',
  '02310103003',
  '02310121230',
  '02310102222',
  '02310122213',
  '02310031110',
  '02132322233',
  '02310011111',
  '02310101320',
  '02310031300',
  '02310010300',
  '02310102000',
  '02310100211',
  '02310031103',
  '02310033301',
  '02310123203',
  '02310100133',
  '02310122012',
  '02310010332',
  '02310032302',
  '02310100013',
  '02310122001',
  '02310011033',
  '02310031101',
  '02310122211',
  '02310030312',
  '02310031213',
  '02310012100',
  '02310100231',
  '02310102212',
  '02310100003',
  '02310013013',
  '02310032300',
  '02310100010',
  '02310123011',
  '02310101030',
  '02310122303',
  '02310013012',
  '02310121031',
  '02310100200',
  '02310100310',
  '02310121122',
  '02132323232',
  '02310032130',
  '02310011001',
  '02310103212',
  '02310100323',
  '02310033130',
  '02310102121',
  '02310123000',
  '02310121102',
  '02310031201',
  

In [70]:
print(f" total quadkeys: {len(all_gNatsGo_quadKeys)}")

 total quadkeys: 712


In [66]:
all_gNatsGo_quadKeys[500]

'02310200021'

In [71]:
#checking key lat long 
qk = all_gNatsGo_quadKeys[0]
get_bounding_lng_lat(qk)

[-111.97265625, -111.796875, 28.149503211544573, 27.99440141104615]

In [72]:
#checking key lat long 
qk = all_gNatsGo_quadKeys[300]
get_bounding_lng_lat(qk)

[-112.1484375, -111.97265625, 31.952162238024968, 31.80289258670676]

In [73]:
#checking key lat long 
qk = all_gNatsGo_quadKeys[700]
get_bounding_lng_lat(qk)

[-111.4453125, -111.26953125, 40.044437584608566, 39.90973623453718]

In [ ]:
31.80289258670676,-112.1484375